In [1]:
from transformers import AutoModelForCausalLM
from peft import get_peft_config,get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, LoraConfig, TaskType, peft_model_load_and_dispatch
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator,get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/bloomz-7b1"
tokenizer_name_or_path = "bigscience/bloomz-7b1"
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

dataset_name = "twitter_complaints"
checkpoint_name = "/home/sourab/"+f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace("/", "_")
text_column = "Tweet text"
label_column = "text_label"
max_length=64
lr = 1e-3
num_epochs = 50
batch_size=8


In [2]:
from datasets import load_dataset

dataset = load_dataset("ought/raft", dataset_name)

classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
    
)
print(dataset)
dataset["train"][0]

Found cached dataset raft (/home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-05388978db6af01d.arrow
Loading cached processed dataset at /home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-e3fade69c4ae889a.arrow


['Unlabeled', 'complaint', 'no complaint']
DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [3]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        #print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids 
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    #print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(max_length-len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0]*(max_length-len(sample_input_ids)) + model_inputs["attention_mask"][i]
        labels["input_ids"][i] =  [-100]*(max_length-len(sample_input_ids)) + label_input_ids 
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length]) 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



processed_datasets = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
        )

train_dataset = processed_datasets["train"]


train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
    )



    

3


Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

In [4]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    #print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(max_length-len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0]*(max_length-len(sample_input_ids)) + model_inputs["attention_mask"][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs

processed_datasets = dataset.map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
        )

eval_dataset = processed_datasets["train"]
test_dataset = processed_datasets["test"]

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
print(next(iter(eval_dataloader)))
print(next(iter(test_dataloader)))

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3, 227985,   5484,    915,   2566, 169403,  15296,  36272,
            525,   3928,   1119,    632,   2670,   3968,  15270,  77658,    915,
            210],
        [     3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3, 227985,   5484,    915,   2566,  88653,   2321, 144017, 138861,
          59283,   1152,    613,   2632,  12120,      4,   5673,   1152,  321

In [5]:
next(iter(train_dataloader))

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3, 227985,   5484,    915,   2566,
           31934,   5227,   6640,  16261,  87843,     17, 130462,   9600, 169668,
              28,  13604, 112581,     40,  77658,    915,    210,   1936, 106863,
               3],
         [     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3, 

In [6]:
len(test_dataloader)

425

In [7]:
max_memory={0: "1GIB", 1: "1GIB", 2: "2GIB", 3: "10GIB", "cpu":"30GB"}

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", max_memory=max_memory)
peft_model_load_and_dispatch(model, torch.load(checkpoint_name), peft_config, max_memory)



/home/sourab/pet/src/pet/tuners/lora.py:106: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


trainable params: 3932160 || all params: 7072948224 || trainable%: 0.055594355783029126


PETModelForCausalLM(
  (base_model): LoRAModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 4096)
        (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0): BloomBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): MergedLinear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=16, bias=False)
                (lora_B): Conv1d(16, 8192, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (dense): Linear(in_features=4096, out_features=4096, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_layernorm):

In [8]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): MergedLinear(
            in_features=4096, out_features=12288, bias=True
            (lora_dropout): Dropout(p=0.1, inplace=False)
            (lora_A): Linear(in_features=4096, out_features=16, bias=False)
            (lora_B): Conv1d(16, 8192, kernel_size=(1,), stride=(1,), groups=2, bias=False)
          )
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=40

In [9]:
model.hf_device_map

{'transformer.word_embeddings': 3,
 'lm_head': 3,
 'transformer.word_embeddings_layernorm': 3,
 'transformer.h.0': 3,
 'transformer.h.1': 3,
 'transformer.h.2': 3,
 'transformer.h.3': 3,
 'transformer.h.4': 3,
 'transformer.h.5': 3,
 'transformer.h.6': 3,
 'transformer.h.7': 3,
 'transformer.h.8': 'cpu',
 'transformer.h.9': 'cpu',
 'transformer.h.10': 'cpu',
 'transformer.h.11': 'cpu',
 'transformer.h.12': 'cpu',
 'transformer.h.13': 'cpu',
 'transformer.h.14': 'cpu',
 'transformer.h.15': 'cpu',
 'transformer.h.16': 'cpu',
 'transformer.h.17': 'cpu',
 'transformer.h.18': 'cpu',
 'transformer.h.19': 'cpu',
 'transformer.h.20': 'cpu',
 'transformer.h.21': 'cpu',
 'transformer.h.22': 'cpu',
 'transformer.h.23': 'cpu',
 'transformer.h.24': 'cpu',
 'transformer.h.25': 'cpu',
 'transformer.h.26': 'cpu',
 'transformer.h.27': 'cpu',
 'transformer.h.28': 'cpu',
 'transformer.h.29': 'cpu',
 'transformer.ln_f': 'cpu'}

In [10]:
model.eval()
eval_preds = []
for _, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    eval_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))

  0%|                                                                         | 0/7 [00:00<?, ?it/s]/home/sourab/transformers/src/transformers/generation/utils.py:1306: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████| 7/7 [01:45<00:00, 15.09s/it]


In [11]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["train"][label_column]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=}")
print(f"{eval_preds[:10]=}")
print(f"{dataset['train'][label_column][:10]=}")


accuracy=100.0
eval_preds[:10]=['no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint', 'no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint']
dataset['train'][label_column][:10]=['no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint', 'no complaint', 'no complaint', 'complaint', 'complaint', 'no complaint']


In [12]:
model.eval()
test_preds = []

for _, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    test_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))
    if len(test_preds)>100:
        break
test_preds

  3%|█▋                                                          | 12/425 [03:16<1:52:48, 16.39s/it]


['no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no complaint',
 'no compl

In [13]:
model.eval()
i = 80
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")
print(dataset["test"][i]["Tweet text"])
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))


@sho_help trying to watch Showtime on my iPad, keep getting network issues. Hulu and Netflix are working fine. Help?
{'input_ids': tensor([[227985,   5484,    915,   2566, 125474, 168916,  10343,    427,  24985,
          33642,  14167,    664,   2670,  99607,     15,  11874,  15980,  14218,
          17327,     17, 162966,    530,  59557,   1306,  10789,  15977,     17,
          98135,     34,  77658,    915,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[227985,   5484,    915,   2566, 125474, 168916,  10343,    427,  24985,
          33642,  14167,    664,   2670,  99607,     15,  11874,  15980,  14218,
          17327,     17, 162966,    530,  59557,   1306,  10789,  15977,     17,
          98135,     34,  77658,    915,    210,  16449,   5952,      3,      3,
              3,      3,      3,      3,      3,      3]])
['Tweet text : @sho_help trying to watch Showtime on my 